<a href="https://colab.research.google.com/github/csw180/romanticpy/blob/main/W_pattern_fnder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 바닥신호를 잡기 위한  W 패턴 인식하는 로직

In [ ]:
import numpy as np
import pandas_datareader.data as web
import pandas as pd
import datetime

start = datetime.datetime(2021,10,1)
end = datetime.datetime(2022,2, 24)
# samsung_df = web.DataReader('005930', 'naver', start='2021-10-01', end='2022-02-24')
# samsung_df = web.DataReader('005930', 'naver',start='2021-01-01')
df = web.DataReader('145720', 'naver')
# samsung_df = web.DataReader('006890', 'naver')

df['ma5'] = df['Close'].rolling(window=5).mean()
df['ma20'] = df['Close'].rolling(window=20).mean()
df['vma20'] = df['Volume'].rolling(window=20).mean()
df['ma60'] = df['Close'].rolling(window=60).mean()
df['ma120'] = df['Close'].rolling(window=120).mean()
df['ma240'] = df['Close'].rolling(window=240).mean()

df['Open'] = df['Open'].astype(float, errors = 'raise')
df['High'] = df['High'].astype(float, errors = 'raise')
df['Low'] = df['Low'].astype(float, errors = 'raise')
df['Close'] = df['Close'].astype(float, errors = 'raise')
df['Volume'] = df['Volume'].astype(float, errors = 'raise')

way_cond1 = (df['Close'] > df['ma5']) & \
          (df['Close'].shift(1) < df['ma5'].shift(1)) & \
          (df['Close'].shift(2) < df['ma5'].shift(2)) & \
          (df['Close'].shift(3) < df['ma5'].shift(3))
way_cond2 = (df['Close'] < df['ma5']) &\
          (df['Close'].shift(1) > df['ma5'].shift(1)) &\
          (df['Close'].shift(2) > df['ma5'].shift(2)) &\
          (df['Close'].shift(3) > df['ma5'].shift(3))


df['way'] = np.where( way_cond1,'up', '')
df['way'] = np.where( way_cond2,'down', '')
# df['angle'] = np.where( (df['way'] == 'up'),df['Low'].rolling(3).min(),'' )
# df['angle'] = np.where( (df['way'] == 'down'),df['High'].rolling(3).max(),'' )
# df = df[df['way'] =='up']
# df = df.dropna()
# df.info()
print(df.head(40))


In [ ]:
import numpy as np
df['up'] = np.where( (df['Close'] > df['ma5']) & \
          (df['Close'].shift(1) < df['ma5'].shift(1)) & \
          (df['Close'].shift(2) < df['ma5'].shift(2)) & \
          (df['Close'].shift(3) < df['ma5'].shift(3)),'up', '')
df['down'] = np.where( (df['Close'] < df['ma5']) &\
          (df['Close'].shift(1) > df['ma5'].shift(1)) &\
          (df['Close'].shift(2) > df['ma5'].shift(2)) &\
          (df['Close'].shift(3) > df['ma5'].shift(3)),'down', '')
df['angle'] = np.where( (df['up'] == 'up'),df['Low'].rolling(3).min(),df['High'].rolling(3).max() )


In [ ]:
df_temp  = df[::-1]
df_temp = df_temp.dropna()
print(df_temp)
# df_temp=df[:0]
# df.head(30)
# df_temp.head(30)